# Importing data from TASK1


In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk import WordNetLemmatizer
import re
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
  
url='https://raw.githubusercontent.com/srk-practicum/2022-knu-nlp/kachaikin_branch/Task1_Kachaikin/root2ai%20-%20Data.csv'
df = pd.read_csv(url)

U=list(df['Target'])
A = list(set(df['Target']))

#Let's chose 2 very different labels, for example Bigdata and Cyber Security

V = list(df['Text'])

R = [[V[i],U[i]] for i in range(len(U))]
new_frame = [i for i in R if i[1]== 'Bigdata' or i[1]== 'Cyber Security']
new_frame_texts = [i[0] for i in new_frame]
new_frame_labels = [i[1] for i in new_frame]
new_frame_labels_asreal = []
for i in new_frame_labels:
    if i == 'Bigdata':
      new_frame_labels_asreal.append(1.0)
    elif i == 'Cyber Security':
      new_frame_labels_asreal.append(0.0)
# Okey, now we have new dataframe with labels Bigdata = 1.0 and Cyber Security = 0.0
df = pd.DataFrame(data = {'Text':new_frame_texts , 'Target':new_frame_labels_asreal})

#Let's watch how distributed labels
number_of_bd = 0
number_of_cs = 0
for i in list(df['Target']):
  if float(i) == 1.0:
    number_of_bd+=1
  elif float(i) == 0.0:
    number_of_cs +=1

text1 = ''
for i in new_frame_texts:
  text1+=i

U = list(df['Text'])
lemmatizer = nltk.WordNetLemmatizer()
V = list(df['Target'])

U_f = []
for j in U:
  R = nltk.word_tokenize(j)
  Y = [lemmatizer.lemmatize(i) for i in R]
  StrY = " ".join(Y)
  U_f.append(StrY)

stop_words = stopwords.words("english")

def cleaning(data):
    
    filtered = re.sub("@\S+", " ", data)  # remove mentions
    filtered = re.sub("https*\S+", " ", filtered) # remove url
    filtered = re.sub("#\S+", " ", filtered) # remove hashtags
    filtered = re.sub("\d", " ", filtered) # remove all numbers
    filtered = re.sub('[%s]+' % re.escape(string.punctuation), ' ', filtered)  # remove punctuation
    filtered = re.sub('\n', ' ', filtered) # remove new lines       
    filtered = re.sub('\s{2,}',' ', filtered) # remove extra spaces
    
    filtered = filtered.lower()
    
    filtered = ' '.join([word for word in filtered.split(' ') if word not in stop_words])
    
    return filtered

df = pd.DataFrame(data = {'Text':[cleaning(i) for i in U_f], 'Target':V})

text_dataset = np.array(df['Text'])
label_dataset = np.array(df['Target'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import numpy as np
from sklearn.model_selection import train_test_split


text_dataset_train, text_dataset_test, label_dataset_train, label_dataset_test = train_test_split(df['Text'], 
                                                    df['Target'], test_size=0.1 ,
                                                    random_state=42)
text_dataset_train, text_dataset_test, label_dataset_train, label_dataset_test = np.array(text_dataset_train), np.array(text_dataset_test), np.asarray(label_dataset_train).astype('int32'), np.asarray(label_dataset_test).astype('int32')


# BERT

In [18]:
# !pip3 install transformers - that command we use for installing transformers to colab
import random
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize(sequence):
    tokens = tokenizer.encode_plus(sequence, max_length=128,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

num_of_elements = len(text_dataset_train)

Xids_train = np.zeros((num_of_elements, 128))
Xmask_train = np.zeros((num_of_elements, 128))

for i, sequence in enumerate(text_dataset_train):
    tokens = tokenize(sequence)
    Xids_train[i, :], Xmask_train[i, :] = tokens[0], tokens[1]

Xids_train.shape


(4416, 128)

In [ ]:
from transformers import TFAutoModel
import tensorflow as tf
from tensorflow import keras
from keras import layers

bert = TFAutoModel.from_pretrained('bert-base-cased')

input_ids = tf.keras.layers.Input(shape=(128,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(128,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)

# Classifier head
x = tf.keras.layers.Dense(16, activation ='relu')(embeddings)
y = tf.keras.layers.Dense(1, activation ='sigmoid', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.summary()
history = model.fit(
    [Xids_train, Xmask_train], label_dataset_train,
    validation_split=0.2,
    batch_size = 1024,
    epochs=1)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         